In [1]:
%reset -f
%matplotlib qt
import sys
import mne
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

pick_ch=['P9','P7','P5','P3','PO7','PO3','O1','P10','P8','P6','P4','PO8','PO4','O2']
#%load_ext autoreload
#%autoreload 2
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb
mne.__version__
mne.set_log_level("WARNING")

infolder, outfolder = find_folder()
subs=['02','03','04','05','06','07','08','09']

In [2]:
%run EEG_auxiliary_module.ipynb
data=list()
labels = list()
trials=4
for j in subs:
    fname = op.join(outfolder,'S'+j+'_perc-epo.fif')
    epochs = mne.read_epochs(fname)
    #trials = len(epochs['101']) # average all trials.
    blocks=len(epochs['101'])/trials
    epochs = block_average(epochs, trials, blocks, kind='perc')
    X, Y = convert_epochs_to_2D_array(epochs)
    data.append(X)
    labels.append(Y)

Starting averaging
No zscoring performed
Starting averaging
No zscoring performed
Starting averaging
No zscoring performed
Starting averaging
No zscoring performed
Starting averaging
No zscoring performed
Starting averaging
No zscoring performed
Starting averaging
No zscoring performed


In [ ]:
import itertools
import numpy as np
from sklearn.preprocessing import scale
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

combs = list(itertools.combinations(np.unique(labels[0]), 2))
svm = LinearSVC(C=1)
score = list()
for i,j in enumerate(subs):
    print(j)
    this_X = data[i]
    this_Y = labels[i]
    data_temp = data.copy()
    data_temp.pop(i)
    other_X = np.concatenate(data_temp)
    labels_temp = labels.copy()
    labels_temp.pop(i)
    other_Y = np.concatenate(labels_temp)
    this_X_shape = this_X.shape
    X_temp = np.vstack((this_X,other_X))
    X_temp = scale(X_temp)
    this_X = X_temp[:this_X_shape[0],:]
    other_X = X_temp[this_X_shape[0]:,:]
    comb_score = list()
    for idx,comb in enumerate(combs):
        print(comb)
        this_X_temp_a = this_X[this_Y==comb[0],:]
        this_X_temp_b = this_X[this_Y==comb[1],:]
        this_Y_temp_a = this_Y[this_Y==comb[0]]
        this_Y_temp_b = this_Y[this_Y==comb[1]]
        other_X_temp = other_X[np.logical_or(other_Y==comb[0], other_Y==comb[1]),:]
        other_Y_temp = other_Y[np.logical_or(other_Y==comb[0], other_Y==comb[1])]
        cv_score=list()
        for m in range(this_X_temp_a.shape[0]):
            data_train_a = this_X_temp_a
            data_train_a = np.delete(data_train_a, m, axis=0)
            data_train_b = this_X_temp_b
            data_train_b = np.delete(data_train_b, m, axis=0)
            data_train = np.vstack((data_train_a,data_train_b,other_X_temp))
            targets_train_a = this_Y_temp_a
            targets_train_a = np.delete(targets_train_a, m)
            targets_train_b = this_Y_temp_b
            targets_train_b = np.delete(targets_train_b, m)
            targets_train = np.concatenate([targets_train_a,targets_train_b,other_Y_temp])
            data_test_a = this_X_temp_a[m,:]
            data_test_b = this_X_temp_b[m,:]
            data_test = np.vstack((data_test_a,data_test_b))
            targets_test=comb
            svm.fit(data_train, targets_train)
            cv_score.append(svm.score(data_test, targets_test))
            update_progress((i*idx+m) / (len(combs)*blocks*len(subs)))
        comb_score.append(cv_score)
    score.append(comb_score)

Progress: [#-------------------] 3.5%
(129, 150)
